In [1]:
import os, numpy as np, itertools, pandas as pd, glob, sys
from numpy import array
from PIL import Image
from natsort import natsorted
from dotmap import DotMap
from os.path import join as pjoin, exists as pexists, basename, dirname
from scipy.spatial import distance

PROJECT_ROOT = os.path.dirname(os.getcwd())
sys.path.append(f'{PROJECT_ROOT}/self_super_reconst')

my_basename = lambda s: os.path.splitext(basename(s))[0]
chained = lambda l: list(itertools.chain(*l))
identity = lambda x: x

In [2]:
# Paths/Params
results_path          = f'{PROJECT_ROOT}/results/'             # Experiments' output reconstructions
imagenet_depth_folder = f'{PROJECT_ROOT}/data/imagenet_depth'  # Depth images extracted for ImageNet data
cuda_dev_id           = 0                                      # CUDA device ID

In [3]:
os.environ['CUDA_VISIBLE_DEVICES'] = str(cuda_dev_id)
from torchvision.datasets import ImageFolder
import torch
import torch.nn as nn

use_cuda = os.environ['CUDA_VISIBLE_DEVICES'] != ''

class WithDepth(torch.utils.data.Dataset):
    def __init__(self, dataset_master, dataset_depth_name, depth_only=False):
        super(WithDepth, self).__init__()
        self.dataset_master = dataset_master
        self.depth_only = depth_only
        self.depth_images_paths = natsorted(glob.glob(pjoin(imagenet_depth_folder, dataset_depth_name, '*')))
        assert len(self.dataset_master) == len(self.depth_images_paths), "Cannot resolve correspondence of rgb images and depth maps (diff length)."
        
    def __getitem__(self, index):
        # IMPORTANT: assume correspondence between the RGB images and their depth maps
        img_rgb, class_label = self.dataset_master[index]
        img_depth = array(Image.open(self.depth_images_paths[index]))
        if self.depth_only:
            img = Image.fromarray(img_depth)
        else:
            img_rgb = array(img_rgb)
            img = Image.fromarray(dstack([img_rgb, img_depth[...,None]]))
        return img, class_label

    def __len__(self):
        return len(self.dataset_master)
        

In [4]:
from self_super_reconst.utils import (MultiBranch, perceptual_loss_layer, 
                                      norm_depth_img, cprintm, cprintc, cprint1, 
                                      timeit)
import torchvision.models as torchvis_models
from torchvision.transforms.functional import to_tensor
from self_super_reconst import lpips
import torch

class PerceptualDistance(nn.Module):
    def __init__(self, net='alex', is_lpips=True):
        super().__init__()
        self.net = lpips.LPIPS(net=net, lpips=is_lpips)
    
    def forward(self, img_A, img_B):
        imgs_in = [lpips.im2tensor(array(img)) for img in [img_A, img_B]]
        if use_cuda:
            imgs_in = [img.cuda() for img in imgs_in]
        return self.net.forward(*imgs_in).cpu().detach().squeeze().numpy().item()
    
class DepthPerceptualDistance(nn.Module):
    def __init__(self):
        super().__init__()
        bbn = torchvis_models.__dict__['vgg16'](pretrained=True)
        bbn.features = nn.Sequential(
            nn.Conv2d(1, 64, 3, padding=1), 
            *bbn.features[1:])
        ckpt_name = 'vgg16_depth_only_norm_within_img'

        cprint1('   >> Loading checkpoint: {}'.format(ckpt_name))
        state_dict_loaded = torch.load(f'{PROJECT_ROOT}/data/imagenet_rgbd/{ckpt_name}_best.pth.tar', map_location='cpu')['state_dict']
        state_dict_loaded = { k.replace('module.', ''): v for k, v in state_dict_loaded.items() }
        bbn.load_state_dict(state_dict_loaded)

        branch_dict = {  # VGG16 Blocks  # selectedLayers = [3, 6, 10, 14, 18]
                        # After maxpools
                        'conv1': ['features.{}'.format(i) for i in range(5)],
                        'conv2': ['features.{}'.format(i) for i in range(10)],
                        'conv3': ['features.{}'.format(i) for i in range(17)],
                        'conv4': ['features.{}'.format(i) for i in range(24)],
                        'conv5': ['features.{}'.format(i) for i in range(31)],
                    }

        main_branch = branch_dict['conv5']
        branch_dict = {layer: branch_module_list[-1] for layer, branch_module_list in branch_dict.items()}
        self.feats_extractor = MultiBranch(bbn, branch_dict, main_branch, spatial_out_dims=None)
    
    def extract_feats(self, img):
        img_norm = norm_depth_img(img)
        if use_cuda:
            img_norm = img_norm.cuda()
        return self.feats_extractor(img_norm)
    
    def forward(self, img_A, img_B):
        imgs_in = [norm_depth_img(to_tensor(img).unsqueeze(0)) for img in [img_A, img_B]]
        if use_cuda:
            imgs_in = [img.cuda() for img in imgs_in]
        feats_list = [self.feats_extractor(img) for img in imgs_in]
        return np.mean([perceptual_loss_layer(*feats_layer_AB).cpu().detach().squeeze().numpy().item() for feats_layer_AB in zip(*feats_list)])
    
    def forward_all(self, img_A_list, img_B):
        img_batch = torch.stack([norm_depth_img(to_tensor(img_A)) for img_A in img_A_list])
        img_B = norm_depth_img(to_tensor(img_B).unsqueeze(0))
        if use_cuda:
            img_batch = img_batch.cuda()
            img_B = img_B.cuda()
        feats_A_all = self.feats_extractor(img_batch)
        feats_B = self.feats_extractor(img_B)
        distances = []
        for i in range(len(img_A_list)):
            feats_A = [feats[i] for feats in feats_A_all]
            distances.append(np.mean([perceptual_loss_layer(*feats_layer_AB).cpu().detach().squeeze().numpy().item() for feats_layer_AB in zip(feats_A, feats_B)]))
        
        return distances

# === Distance metrics
distance_metric_dict = {
    'perceptual_alex_cal': PerceptualDistance('alex', True),
    'perceptual_vgg_cal': PerceptualDistance('vgg', True),
    'perceptual_alex_noncal': PerceptualDistance('alex', False),
    'perceptual_vgg_noncal': PerceptualDistance('vgg', False),
    'corr_dist': lambda x, y: distance.correlation(array(x).flatten(), array(y).flatten()),

    'perceptual_vgg_noncal_depth': DepthPerceptualDistance(),
}

# === Distractors
valdir = f'{PROJECT_ROOT}/data/imagenet/val'
distractors_rgb = ImageFolder(valdir)
distractors_depth = WithDepth(distractors_rgb, 'val_depth_on_orig_large_png_uint8', depth_only=True)

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /net/mraid11/export/data/guyga/PycharmProjects/SelfSuperReconst_public/self_super_reconst/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]
Loading model from: /net/mraid11/export/data/guyga/PycharmProjects/SelfSuperReconst_public/self_super_reconst/lpips/weights/v0.1/vgg.pth
Setting up [baseline] perceptual loss: trunk [alex], v[0.1], spatial [off]
Setting up [baseline] perceptual loss: trunk [vgg], v[0.1], spatial [off]
   >> Loading checkpoint: vgg16_depth_only_norm_within_img


In [5]:
# Sort by perceptual similarity

def reorder(file_paths, str_key='depth'):
    return [fp for fp in file_paths if str_key not in my_basename(fp)] + [fp for fp in file_paths if str_key in my_basename(fp)]

isgray = lambda img: array(img).ndim == 2

NO_DEPTH, DEPTH, DEPTH_FROM_TESTAVG = range(3)

eval_opts_defaults = dict(distance_metric_name='perceptual_alex_cal', eval_depth=NO_DEPTH, collapse_func='', n_way=2, repeats=100, rank=0)

def release_cuda(distance_metric):
    if isinstance(distance_metric, nn.Module):
        distance_metric.cpu()
    
@timeit
def eval(exp_name, eval_opts):
    opts = DotMap(dict(eval_opts_defaults))
    opts.update(eval_opts)

    distance_metric = distance_metric_dict[opts.distance_metric_name]
    if isinstance(distance_metric, nn.Module):
        if use_cuda:
            distance_metric.cuda()
        distance_metric.eval()

    collapse_func = getattr(np, opts.collapse_func) if opts.collapse_func else identity
    
    exp_path = pjoin(results_path, exp_name)
    if not pexists(exp_path):
        cprint('(!) Experiment not found: {}'.format(exp_path), 'red')
        return
    
    if opts.eval_depth == DEPTH_FROM_TESTAVG:
        paths_list = natsorted(glob.glob(pjoin(exp_path, 'depth_from_test_avg', '*')))
    else:
        paths_list = natsorted(glob.glob(pjoin(exp_path, 'test_avg', '*')))

    if opts.eval_depth == NO_DEPTH:
        paths_list = [p for p in paths_list if not my_basename(p).endswith('depth')]
    else:
        paths_list = [p for p in paths_list if my_basename(p).endswith('gray_depth')]

    sbs_list = [lpips.load_image(p) for p in paths_list]
        
    n_chan = 1 if opts.eval_depth else 3
    img_list_actual, img_list_pred = zip(*[(Image.fromarray(img[:, :112, :n_chan].squeeze()), Image.fromarray(img[:, 112:, :n_chan].squeeze())) for img in sbs_list])
        
    if opts.eval_depth:
        assert isgray(img_list_actual[0]) and isgray(img_list_pred[0])
        
    with torch.no_grad():
        if opts.n_way > 0:
            if opts.eval_depth:
                distractor_list = distractors_depth
            else:
                distractor_list = distractors_rgb
            results = []
            results_rank = []
            for img_actual, img_pred in zip(img_list_actual, img_list_pred):
                results_img = []
                results_img_rank = []
                for _ in range(opts.repeats):
                    cands = [img_actual] + [distractor_list[ii][0].resize(img_actual.size, Image.ANTIALIAS) for ii in np.random.permutation(len(distractor_list))[:opts.n_way - 1]]
                        
                    if getattr(distance_metric, "forward_all", None):
                        distances = distance_metric.forward_all(cands, img_pred)
                    else:
                        distances = [distance_metric(img_cand, img_pred) for img_cand in cands]
                    
                    results_img_rank.append(np.argwhere(np.argsort(distances) == 0).flatten()[0] / (len(distances) - 1))
                    results_img.append(np.argsort(distances)[0] == 0)

                results_rank.append(np.mean(results_img_rank))
                results.append(np.mean(results_img))
            release_cuda(distance_metric)
            return collapse_func(results), collapse_func(results_rank)
        else:  # Pairwise evaluation
            res = collapse_func([distance_metric(array(img_actual), array(img_pred)) for img_actual, img_pred in zip(img_list_actual, img_list_pred)])
            release_cuda(distance_metric)
            return res

In [6]:
files_list = [x for x in glob.glob(results_path + '/*') if pexists(x + '/test_avg')]
files_list_sorted = sorted(files_list, key=os.path.getctime)
list(map(os.path.basename, files_list_sorted[-10:]))

['sub3_rgbd_noDE', 'sub3_rgb_only_noDE', 'sub3_depth_only_noDE']

In [7]:
exp_list = list(files_list_sorted)
list(map(os.path.basename, exp_list))

['sub3_rgbd_noDE', 'sub3_rgb_only_noDE', 'sub3_depth_only_noDE']

### A few examples of evaluations

In [8]:
comparison_prog = []

comparison_prog += [
    dict(
        exp_name=exp_name, 
        opts=chained([
            [
                dict(n_way=n_way, eval_depth=NO_DEPTH, distance_metric_name='perceptual_vgg_noncal', collapse_func='', rank=0, repeats=10),
                dict(n_way=n_way, eval_depth=NO_DEPTH, distance_metric_name='perceptual_vgg_noncal', collapse_func='', rank=1, repeats=10),
            ] for n_way in [5, 10]  # , 50, 100, 500, 1000]
        ])
    ) for exp_name in chained([
        [
            f'sub{sbj_num}_rgbd_noDE', 
            f'sub{sbj_num}_rgb_only_noDE',
#             f'sub{sbj_num}_rgbd', 
#             f'sub{sbj_num}_rgb_only',
#         ] for sbj_num in range(1,6)])
        ] for sbj_num in [3]])
    ]

comparison_prog += [
    dict(
        exp_name=exp_name, 
        opts=chained([
            [
                dict(n_way=n_way, eval_depth=DEPTH, distance_metric_name='perceptual_vgg_noncal_depth', collapse_func='', rank=0, repeats=10),
                dict(n_way=n_way, eval_depth=DEPTH, distance_metric_name='perceptual_vgg_noncal_depth', collapse_func='', rank=1, repeats=10),
            ] for n_way in [5, 10]  # , 50, 100, 500, 1000]
        ])
    ) for exp_name in chained([
        [
            f'sub{sbj_num}_rgbd_noDE', 
            f'sub{sbj_num}_depth_only_noDE',
#             f'sub{sbj_num}_rgb_only_noDE',  # (See decoder flag 'depth_from_rgb' which could make depth evaluation here possible)
#             f'sub{sbj_num}_rgbd', 
#             f'sub{sbj_num}_depth_only',
#             f'sub{sbj_num}_rgb_only',
#         ] for sbj_num in range(1,6)])
        ] for sbj_num in [3]])
    ]

# comparison_prog

In [9]:
# Load previously evaluated data and add new evaluations
# df = pd.read_pickle('eval_results/eval.pkl')

# Or create a new one
df = pd.DataFrame()

df

""


In [10]:
# Add new evaluation requests
data = { 
    **{ 'exp_name': [], 'score': [] }, 
    **{ k: [] for k in eval_opts_defaults} 
    }

for comparison_prog_exp in comparison_prog:
    for opts_dict_delta in comparison_prog_exp['opts']:
        opts_dict = dict(eval_opts_defaults)
        opts_dict.update(opts_dict_delta)
        data['exp_name'].append(comparison_prog_exp['exp_name'])
        for opt, opt_val in opts_dict.items():
            data[opt].append(opt_val)
        data['score'].append('')
           
df1 = pd.DataFrame.from_dict(data)

df = pd.concat([df, df1]).drop_duplicates(subset=['exp_name'] + list(eval_opts_defaults.keys())).reset_index(drop=True)
df

,exp_name,score,distance_metric_name,eval_depth,collapse_func,n_way,repeats,rank
0,sub3_rgbd_noDE,,perceptual_vgg_noncal,0,,5,10,0
1,sub3_rgbd_noDE,,perceptual_vgg_noncal,0,,5,10,1
2,sub3_rgbd_noDE,,perceptual_vgg_noncal,0,,10,10,0
3,sub3_rgbd_noDE,,perceptual_vgg_noncal,0,,10,10,1
4,sub3_rgb_only_noDE,,perceptual_vgg_noncal,0,,5,10,0
5,sub3_rgb_only_noDE,,perceptual_vgg_noncal,0,,5,10,1
6,sub3_rgb_only_noDE,,perceptual_vgg_noncal,0,,10,10,0
7,sub3_rgb_only_noDE,,perceptual_vgg_noncal,0,,10,10,1
8,sub3_rgbd_noDE,,perceptual_vgg_noncal_depth,1,,5,10,0
9,sub3_rgbd_noDE,,perceptual_vgg_noncal_depth,1,,5,10,1


### Run evaluations

In [11]:
skip_indices = []
for i, row in df.iterrows():
    if row.score != '' or i in skip_indices: 
        continue
    row.pop('score')
    exp_name = row.pop('exp_name')
    print('=' * 20)
    cprint1(exp_name)
    print('\n'.join(str(row).split('\n')[:-1]))
    res = eval(exp_name, dict(row))
    if res == None:
        continue
    if not isinstance(res, list) and type(res) != tuple:
        res = res.round(3)
    if type(res) == tuple:
        filter_v = dict(row)
        filter_v.update({'exp_name': exp_name, 'score': ''})
        if dict(row)['rank'] == 0:
            filter_v.update({'rank': 1})
            j = df.index[(df[list(filter_v)] == pd.Series(filter_v)).all(axis=1)]
            if len(j):
                assert len(j) == 1
                j = j[0]
                df.at[j, 'score'] = res[1]
                skip_indices.append(j)
            res = res[0]
        else:  # rank == 1
            filter_v.update({'rank': 0})
            j = df.index[(df[list(filter_v)] == pd.Series(filter_v)).all(axis=1)]
            if len(j):
                assert len(j) == 1
                j = j[0]
                df.at[j, 'score'] = res[0]
                skip_indices.append(j)
            res = res[1]

    df.at[i, 'score'] = res
cprintm('Done')



sub3_rgbd_noDE
distance_metric_name    perceptual_vgg_noncal
eval_depth                                  0
collapse_func                                
n_way                                       5
repeats                                    10
rank                                        0
eval | 20.5 sec
sub3_rgbd_noDE
distance_metric_name    perceptual_vgg_noncal
eval_depth                                  0
collapse_func                                
n_way                                      10
repeats                                    10
rank                                        0
eval | 36.6 sec
sub3_rgb_only_noDE
distance_metric_name    perceptual_vgg_noncal
eval_depth                                  0
collapse_func                                
n_way                                       5
repeats                                    10
rank                                        0
eval | 17.8 sec
sub3_rgb_only_noDE
distance_metric_name    perceptual_vgg_noncal
eval_depth

In [12]:
# Save to evaluation file
df.to_pickle(f'{PROJECT_ROOT}/eval_results/my_eval.pkl')  # Can also override original pkl file